In [ ]:
import sys
import os
import numpy as np
import zipfile
from tqdm import tqdm
import scrapbook as sb
from tempfile import TemporaryDirectory
import tensorflow as tf
tf.get_logger().setLevel('ERROR') # only show error messages

from recommenders.models.deeprec.deeprec_utils import download_deeprec_resources
from recommenders.models.newsrec.newsrec_utils import prepare_hparams
from recommenders.models.newsrec.models.lstur import LSTURModel
from recommenders.models.newsrec.io.mind_iterator import MINDIterator
from recommenders.models.newsrec.newsrec_utils import get_mind_data_set

print("System version:{}".format(sys.version))
print("Tensorflow version:{}".format(tf.__version__))

System version:3.8.17 (default, Jul  5 2023, 20:44:21) [MSC v.1916 64 bit (AMD64)]
Tensorflow version:2.6.1


In [ ]:
import numpy as np

In [ ]:
!pip install keras==2.6

In [ ]:
!pip install protobuf==3.20.* --user

In [ ]:
!pip install scrapbook

In [ ]:
!pip install tensorflow==2.6.1

In [ ]:
!pip install numpy==1.20.3 --user

In [ ]:
conda uninstall numpy

In [ ]:
!pip show numpy

In [2]:
import pandas as pd
import random
import numpy as np

In [3]:
df1=pd.read_table('traces.tsv')
df1

,auction_id,bid_time,bid_ct,bid_number,bid_user,bid_butler,bid_cp,bid_user_secs_added,bid_butler_secs_added,bid_infered,bid_group,bid_final
0,222387,2009-10-04 12:33:26,19913,1,Becker1990,0,12,0,0,0.0,0.0,0.0
1,222387,2009-10-04 14:28:14,13029,2,Gaynor66,0,24,4,0,0.0,1.0,0.0
2,222387,2009-10-04 14:44:08,12079,3,Jostrem16,0,36,4,0,0.0,2.0,0.0
3,222387,2009-10-04 17:14:26,3071,4,Clubfan78,0,48,10,0,0.0,3.0,0.0
4,222387,2009-10-04 17:36:06,1773,5,Momstired2,0,60,2,0,0.0,4.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
270138,228024,2009-10-13 15:14:07,20,174,Teammaine,0,1044,0,0,1.0,113.0,0.0
270139,228024,2009-10-13 15:14:07,20,175,Smcsweeps,0,1050,0,0,0.0,113.0,0.0
270140,228024,2009-10-13 15:14:09,20,176,Xraymandan,0,1056,0,0,1.0,114.0,0.0
270141,228024,2009-10-13 15:14:09,20,177,Palme02,0,1062,0,0,0.0,114.0,0.0


In [4]:
df2=df1.drop_duplicates(subset=['auction_id','bid_user']).groupby('bid_user').size().sort_values(ascending=False).reset_index()
df2.columns =['bid_user', 'counts']
df2

,bid_user,counts
0,Npr1967,101
1,Alad99,78
2,Mogana,66
3,Garand98,64
4,Karejoca,62
...,...,...
11472,Mikedbrooks,1
11473,Mikebggles1,1
11474,Mikeali92,1
11475,Mike6432,1


In [5]:
df2=df2[df2['counts']>=2]
df2

,bid_user,counts
0,Npr1967,101
1,Alad99,78
2,Mogana,66
3,Garand98,64
4,Karejoca,62
...,...,...
6264,Gjmr123,2
6265,Paul7997,2
6266,Rsis67,2
6267,Paulsheppard,2


In [6]:
df3=df1[df1['bid_user'].isin(df2['bid_user'].tolist())]
df3

,auction_id,bid_time,bid_ct,bid_number,bid_user,bid_butler,bid_cp,bid_user_secs_added,bid_butler_secs_added,bid_infered,bid_group,bid_final
0,222387,2009-10-04 12:33:26,19913,1,Becker1990,0,12,0,0,0.0,0.0,0.0
1,222387,2009-10-04 14:28:14,13029,2,Gaynor66,0,24,4,0,0.0,1.0,0.0
2,222387,2009-10-04 14:44:08,12079,3,Jostrem16,0,36,4,0,0.0,2.0,0.0
3,222387,2009-10-04 17:14:26,3071,4,Clubfan78,0,48,10,0,0.0,3.0,0.0
4,222387,2009-10-04 17:36:06,1773,5,Momstired2,0,60,2,0,0.0,4.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
270138,228024,2009-10-13 15:14:07,20,174,Teammaine,0,1044,0,0,1.0,113.0,0.0
270139,228024,2009-10-13 15:14:07,20,175,Smcsweeps,0,1050,0,0,0.0,113.0,0.0
270140,228024,2009-10-13 15:14:09,20,176,Xraymandan,0,1056,0,0,1.0,114.0,0.0
270141,228024,2009-10-13 15:14:09,20,177,Palme02,0,1062,0,0,0.0,114.0,0.0


In [7]:
df4=df3.drop_duplicates(subset=['auction_id','bid_user']).groupby('bid_user').size().sort_values(ascending=False)#reset_index()
df4

bid_user
Npr1967       101
Alad99         78
Mogana         66
Garand98       64
Karejoca       62
             ... 
Oinkie          2
Dodiebob        2
Okitty          2
Okmenow         2
Darren1014      2
Length: 6269, dtype: int64

In [8]:
df5=df3.sort_values('bid_time')
df5=df5.drop_duplicates(subset=['auction_id','bid_user'])
df5

,auction_id,bid_time,bid_ct,bid_number,bid_user,bid_butler,bid_cp,bid_user_secs_added,bid_butler_secs_added,bid_infered,bid_group,bid_final
19003,223829,2009-10-01 22:07:40,2854,1,Sash6,0,12,0,0,0.0,0.0,0.0
19006,223829,2009-10-01 22:51:21,248,4,Rfire,0,48,2,0,0.0,3.0,0.0
19007,223829,2009-10-01 22:52:14,197,5,dome60,0,60,2,0,0.0,4.0,0.0
19014,223829,2009-10-01 22:55:53,14,12,Yllekbryan,0,144,0,0,1.0,9.0,0.0
3924,222653,2009-10-02 03:17:59,36737,1,Rvs66,0,6,0,0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
237219,227753,2009-10-18 20:49:33,20,18,Jingmei,0,216,0,0,0.0,17.0,0.0
237991,227767,2009-10-18 21:31:47,947,2,Mikrod,0,24,15,0,0.0,1.0,0.0
237992,227767,2009-10-18 21:44:16,214,3,Dymgroup,0,36,16,0,0.0,2.0,0.0
237993,227767,2009-10-18 21:44:44,197,4,Chefzac,0,48,11,0,0.0,3.0,0.0


In [9]:
df5['auction_id']=df5['auction_id'].astype(str)

In [10]:
df5

,auction_id,bid_time,bid_ct,bid_number,bid_user,bid_butler,bid_cp,bid_user_secs_added,bid_butler_secs_added,bid_infered,bid_group,bid_final
19003,223829,2009-10-01 22:07:40,2854,1,Sash6,0,12,0,0,0.0,0.0,0.0
19006,223829,2009-10-01 22:51:21,248,4,Rfire,0,48,2,0,0.0,3.0,0.0
19007,223829,2009-10-01 22:52:14,197,5,dome60,0,60,2,0,0.0,4.0,0.0
19014,223829,2009-10-01 22:55:53,14,12,Yllekbryan,0,144,0,0,1.0,9.0,0.0
3924,222653,2009-10-02 03:17:59,36737,1,Rvs66,0,6,0,0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
237219,227753,2009-10-18 20:49:33,20,18,Jingmei,0,216,0,0,0.0,17.0,0.0
237991,227767,2009-10-18 21:31:47,947,2,Mikrod,0,24,15,0,0.0,1.0,0.0
237992,227767,2009-10-18 21:44:16,214,3,Dymgroup,0,36,16,0,0.0,2.0,0.0
237993,227767,2009-10-18 21:44:44,197,4,Chefzac,0,48,11,0,0.0,3.0,0.0


In [11]:
impressionData=df5.groupby('bid_user')['auction_id'].apply(" ".join).reset_index()
print(impressionData)

           bid_user                                         auction_id
0      +34661281XXX                        224875 224929 223626 225739
1     +441479812XXX                               227990 227988 227878
2     +441698293XXX                 225791 225811 225776 225803 225793
3     +441757291XXX                                      225801 227994
4     +441786449XXX                               225803 225760 225773
...             ...                                                ...
6264        waggerl   225844 227524 225470 225757 225449 225577 225901
6265       willich1                 225825 225903 225776 225463 225896
6266      winni1958  224289 224875 227667 227894 227664 227660 2253...
6267      wolfhorst   225648 225892 228016 228017 225742 228022 228024
6268        zarte28  224026 225028 227526 225181 225553 225885 2251...

[6269 rows x 2 columns]


In [12]:
impressionData['auction_id'].dtype

dtype('O')

In [13]:
def noLastOne(text):
  words=text.split()
  if len(words)<=1:
    return ''
  else:
    return ' '.join(words[-51:-1])

def keepLastOne(text):
  newWords=text.split()
  if len(newWords)<1:
    return ''
  else:
     impressionWords=[newWords[-1]]+[str(random.randint(100000, 999999))for _ in range(20)]
     random_index = random.randint(0,len(impressionWords)-1)
     impressionWords[random_index]+='-1'
     return ' '.join(impressionWords)

impressionData['history']=impressionData['auction_id'].apply(noLastOne)
impressionData['impression']=impressionData['auction_id'].apply(keepLastOne)

print(impressionData)

           bid_user                                         auction_id  \
0      +34661281XXX                        224875 224929 223626 225739   
1     +441479812XXX                               227990 227988 227878   
2     +441698293XXX                 225791 225811 225776 225803 225793   
3     +441757291XXX                                      225801 227994   
4     +441786449XXX                               225803 225760 225773   
...             ...                                                ...   
6264        waggerl   225844 227524 225470 225757 225449 225577 225901   
6265       willich1                 225825 225903 225776 225463 225896   
6266      winni1958  224289 224875 227667 227894 227664 227660 2253...   
6267      wolfhorst   225648 225892 228016 228017 225742 228022 228024   
6268        zarte28  224026 225028 227526 225181 225553 225885 2251...   

                                                history  \
0                                  224875 224929 223

In [14]:
impression2=impressionData
impression2

,bid_user,auction_id,history,impression
0,+34661281XXX,224875 224929 223626 225739,224875 224929 223626,225739-1 160320 145816 214131 474704 101274 18...
1,+441479812XXX,227990 227988 227878,227990 227988,227878 859166 377596 707828 289374 655033 9761...
2,+441698293XXX,225791 225811 225776 225803 225793,225791 225811 225776 225803,225793 783219 300805 437643 368991 260490 9410...
3,+441757291XXX,225801 227994,225801,227994 272152 596477 774377 511686 726329 6806...
4,+441786449XXX,225803 225760 225773,225803 225760,225773 360957-1 627339 993258 674175 755488 64...
...,...,...,...,...
6264,waggerl,225844 227524 225470 225757 225449 225577 225901,225844 227524 225470 225757 225449 225577,225901 408900 153273 283245 931825 465467 3673...
6265,willich1,225825 225903 225776 225463 225896,225825 225903 225776 225463,225896 572558 244309 475599 859399 602784 4191...
6266,winni1958,224289 224875 227667 227894 227664 227660 2253...,224289 224875 227667 227894 227664 227660 2253...,225582 294310 988890 494467 601841 510161 2212...
6267,wolfhorst,225648 225892 228016 228017 225742 228022 228024,225648 225892 228016 228017 225742 228022,228024 470190 197460 149050 235570-1 368077 20...


In [51]:
def addZero(text):
    words=text.split()
    newWords=[]
    for word in words:
        if '-1' not in word:
            word+='-0'
        newWords.append(word)

    return ' '.join(newWords)

impression2['impression']=impression2['impression'].apply(addZero)

print(impression2)

           bid_user                                         auction_id  \
0      +34661281XXX                        224875 224929 223626 225739   
1     +441479812XXX                               227990 227988 227878   
2     +441698293XXX                 225791 225811 225776 225803 225793   
3     +441757291XXX                                      225801 227994   
4     +441786449XXX                               225803 225760 225773   
...             ...                                                ...   
6264        waggerl   225844 227524 225470 225757 225449 225577 225901   
6265       willich1                 225825 225903 225776 225463 225896   
6266      winni1958  224289 224875 227667 227894 227664 227660 2253...   
6267      wolfhorst   225648 225892 228016 228017 225742 228022 228024   
6268        zarte28  224026 225028 227526 225181 225553 225885 2251...   

                                                history  \
0                                  224875 224929 223

In [52]:
impression3=impression2.drop('auction_id',axis='columns')
impression3

,bid_user,history,impression
0,+34661281XXX,224875 224929 223626,225739-1 160320-0 145816-0 214131-0 474704-0 1...
1,+441479812XXX,227990 227988,227878-0 859166-0 377596-0 707828-0 289374-0 6...
2,+441698293XXX,225791 225811 225776 225803,225793-0 783219-0 300805-0 437643-0 368991-0 2...
3,+441757291XXX,225801,227994-0 272152-0 596477-0 774377-0 511686-0 7...
4,+441786449XXX,225803 225760,225773-0 360957-1 627339-0 993258-0 674175-0 7...
...,...,...,...
6264,waggerl,225844 227524 225470 225757 225449 225577,225901-0 408900-0 153273-0 283245-0 931825-0 4...
6265,willich1,225825 225903 225776 225463,225896-0 572558-0 244309-0 475599-0 859399-0 6...
6266,winni1958,224289 224875 227667 227894 227664 227660 2253...,225582-0 294310-0 988890-0 494467-0 601841-0 5...
6267,wolfhorst,225648 225892 228016 228017 225742 228022,228024-0 470190-0 197460-0 149050-0 235570-1 3...


In [53]:
impression4=impression3.head(100)
impression4

,bid_user,history,impression
0,+34661281XXX,224875 224929 223626,225739-1 160320-0 145816-0 214131-0 474704-0 1...
1,+441479812XXX,227990 227988,227878-0 859166-0 377596-0 707828-0 289374-0 6...
2,+441698293XXX,225791 225811 225776 225803,225793-0 783219-0 300805-0 437643-0 368991-0 2...
3,+441757291XXX,225801,227994-0 272152-0 596477-0 774377-0 511686-0 7...
4,+441786449XXX,225803 225760,225773-0 360957-1 627339-0 993258-0 674175-0 7...
...,...,...,...
95,24091950,225903 225217,226683-0 995177-0 512820-0 766745-0 668425-0 6...
96,24133313,225844 226151 225896,226554-0 368336-0 394064-0 383055-0 819137-0 6...
97,29061979,226148,228021-0 683662-0 594704-0 833115-0 355760-0 1...
98,2broke2by,226162,226174-0 851438-0 684764-0 820432-0 340256-0 5...


In [54]:
impressionTime=['11/11/2019 9:05:58 AM','11/11/2019 9:05:58 AM','11/11/2019 9:05:58 AM','11/11/2019 9:05:58 AM','11/11/2019 9:05:58 AM',
                '11/11/2019 9:05:58 AM','11/11/2019 9:05:58 AM','11/11/2019 9:05:58 AM','11/11/2019 9:05:58 AM','11/11/2019 9:05:58 AM',
                '11/11/2019 9:05:58 AM','11/11/2019 9:05:58 AM','11/11/2019 9:05:58 AM','11/11/2019 9:05:58 AM','11/11/2019 9:05:58 AM',
                '11/11/2019 9:05:58 AM','11/11/2019 9:05:58 AM','11/11/2019 9:05:58 AM','11/11/2019 9:05:58 AM','11/11/2019 9:05:58 AM',
                '11/11/2019 9:05:58 AM','11/11/2019 9:05:58 AM','11/11/2019 9:05:58 AM','11/11/2019 9:05:58 AM','11/11/2019 9:05:58 AM',
                '11/11/2019 9:05:58 AM','11/11/2019 9:05:58 AM','11/11/2019 9:05:58 AM','11/11/2019 9:05:58 AM','11/11/2019 9:05:58 AM',
                '11/11/2019 9:05:58 AM','11/11/2019 9:05:58 AM','11/11/2019 9:05:58 AM','11/11/2019 9:05:58 AM','11/11/2019 9:05:58 AM',
                '11/11/2019 9:05:58 AM','11/11/2019 9:05:58 AM','11/11/2019 9:05:58 AM','11/11/2019 9:05:58 AM','11/11/2019 9:05:58 AM',
                '11/11/2019 9:05:58 AM','11/11/2019 9:05:58 AM','11/11/2019 9:05:58 AM','11/11/2019 9:05:58 AM','11/11/2019 9:05:58 AM',
                '11/11/2019 9:05:58 AM','11/11/2019 9:05:58 AM','11/11/2019 9:05:58 AM','11/11/2019 9:05:58 AM','11/11/2019 9:05:58 AM',
                '11/11/2019 9:05:58 AM','11/11/2019 9:05:58 AM','11/11/2019 9:05:58 AM','11/11/2019 9:05:58 AM','11/11/2019 9:05:58 AM',
                '11/11/2019 9:05:58 AM','11/11/2019 9:05:58 AM','11/11/2019 9:05:58 AM','11/11/2019 9:05:58 AM','11/11/2019 9:05:58 AM',
                '11/11/2019 9:05:58 AM','11/11/2019 9:05:58 AM','11/11/2019 9:05:58 AM','11/11/2019 9:05:58 AM','11/11/2019 9:05:58 AM',
                '11/11/2019 9:05:58 AM','11/11/2019 9:05:58 AM','11/11/2019 9:05:58 AM','11/11/2019 9:05:58 AM','11/11/2019 9:05:58 AM',
                '11/11/2019 9:05:58 AM','11/11/2019 9:05:58 AM','11/11/2019 9:05:58 AM','11/11/2019 9:05:58 AM','11/11/2019 9:05:58 AM',
                '11/11/2019 9:05:58 AM','11/11/2019 9:05:58 AM','11/11/2019 9:05:58 AM','11/11/2019 9:05:58 AM','11/11/2019 9:05:58 AM',
                '11/11/2019 9:05:58 AM','11/11/2019 9:05:58 AM','11/11/2019 9:05:58 AM','11/11/2019 9:05:58 AM','11/11/2019 9:05:58 AM',
                '11/11/2019 9:05:58 AM','11/11/2019 9:05:58 AM','11/11/2019 9:05:58 AM','11/11/2019 9:05:58 AM','11/11/2019 9:05:58 AM',
                '11/11/2019 9:05:58 AM','11/11/2019 9:05:58 AM','11/11/2019 9:05:58 AM','11/11/2019 9:05:58 AM','11/11/2019 9:05:58 AM',
                '11/11/2019 9:05:58 AM','11/11/2019 9:05:58 AM','11/11/2019 9:05:58 AM','11/11/2019 9:05:58 AM','11/11/2019 9:05:58 AM']
impression4['impressionTime']=impressionTime
impression4.head()

<ipython-input-54-be719d00ef4f>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  impression4['impressionTime']=impressionTime


,bid_user,history,impression,impressionTime
0,+34661281XXX,224875 224929 223626,225739-1 160320-0 145816-0 214131-0 474704-0 1...,11/11/2019 9:05:58 AM
1,+441479812XXX,227990 227988,227878-0 859166-0 377596-0 707828-0 289374-0 6...,11/11/2019 9:05:58 AM
2,+441698293XXX,225791 225811 225776 225803,225793-0 783219-0 300805-0 437643-0 368991-0 2...,11/11/2019 9:05:58 AM
3,+441757291XXX,225801,227994-0 272152-0 596477-0 774377-0 511686-0 7...,11/11/2019 9:05:58 AM
4,+441786449XXX,225803 225760,225773-0 360957-1 627339-0 993258-0 674175-0 7...,11/11/2019 9:05:58 AM


In [55]:
impression4['impressionTime'].dtype

dtype('O')

In [56]:
impression4=impression4[['bid_user','impressionTime','history','impression']]
impression4

,bid_user,impressionTime,history,impression
0,+34661281XXX,11/11/2019 9:05:58 AM,224875 224929 223626,225739-1 160320-0 145816-0 214131-0 474704-0 1...
1,+441479812XXX,11/11/2019 9:05:58 AM,227990 227988,227878-0 859166-0 377596-0 707828-0 289374-0 6...
2,+441698293XXX,11/11/2019 9:05:58 AM,225791 225811 225776 225803,225793-0 783219-0 300805-0 437643-0 368991-0 2...
3,+441757291XXX,11/11/2019 9:05:58 AM,225801,227994-0 272152-0 596477-0 774377-0 511686-0 7...
4,+441786449XXX,11/11/2019 9:05:58 AM,225803 225760,225773-0 360957-1 627339-0 993258-0 674175-0 7...
...,...,...,...,...
95,24091950,11/11/2019 9:05:58 AM,225903 225217,226683-0 995177-0 512820-0 766745-0 668425-0 6...
96,24133313,11/11/2019 9:05:58 AM,225844 226151 225896,226554-0 368336-0 394064-0 383055-0 819137-0 6...
97,29061979,11/11/2019 9:05:58 AM,226148,228021-0 683662-0 594704-0 833115-0 355760-0 1...
98,2broke2by,11/11/2019 9:05:58 AM,226162,226174-0 851438-0 684764-0 820432-0 340256-0 5...


In [57]:
impression4.to_csv('impression4.tsv',header=False,sep="\t")

In [58]:
impression4.to_csv('impression4.csv')

In [22]:
df123=pd.read_table('behaviors.tsv')
df123.head()

,0,+34661281XXX,11/11/2019 9:05:58 AM,224875 224929 223626,225739-0 765347-0 792185-0 781206-0 456401-0 139544-0 443455-1 455492-0 380960-0 488754-0 945136-0 547254-0 717104-0 958030-0 496369-0 390035-0 137939-0 477685-0 990297-0 286729-0 999444-0
0,1,+34938723XXX,11/11/2019 9:05:58 AM,247448,247083-0 407613-0 926952-0 732033-0 740802-0 1...
1,2,+34976394XXX,11/11/2019 9:05:58 AM,239006,240958-0 430335-0 623927-0 436151-0 302521-0 3...
2,3,+436604727XXX,11/11/2019 9:05:58 AM,231525,243429-0 376944-0 725942-0 163020-0 272084-0 5...
3,4,+436606822XXX,11/11/2019 9:05:58 AM,229731 248057 249875,250942-0 627679-0 788393-0 816219-0 603107-0 5...
4,5,+4401482627XXX,11/11/2019 9:05:58 AM,244339,242507-0 747735-0 272052-0 954203-0 262672-0 6...


In [23]:
df123['11/11/2019 9:05:58 AM'].dtype

dtype('O')

In [24]:
df=pd.read_table('outcomes.tsv')
df.head()

,auction_id,product_id,item,desc,retail,price,finalprice,bidincrement,bidfee,winner,placedbids,freebids,endtime_str,flg_click_only,flg_beginnerauction,flg_fixedprice,flg_endprice
0,86827,10009602,sony-ericsson-s500i-unlocked-mysterious-,Sony Ericsson S500i Unlocked Mysterious Green,499.99,13.35,13.35,15,75,Racer11,26,0,19:52 PDT 09-16-2008,0,0,0,0
1,87964,10009881,psp-slim-lite-sony-piano-black,PSP Slim & Lite Sony Piano Black,169.99,74.70,74.70,15,75,Cemo23,65,0,11:17 PDT 08-28-2008,0,0,0,0
2,87965,10009881,psp-slim-lite-sony-piano-black,PSP Slim & Lite Sony Piano Black,169.99,83.10,83.10,15,75,Jacobsonnich,94,0,22:52 PDT 11-07-2008,0,1,0,0
3,88638,10006115,sony-ericsson-s500i-unlocked-mysterious-,Sony Ericsson S500i Unlocked Mysterious Green,499.99,19.65,19.65,15,75,Mokkis,10,0,22:02 PDT 08-23-2008,0,0,0,0
4,88639,10006115,sony-ericsson-s500i-unlocked-mysterious-,Sony Ericsson S500i Unlocked Mysterious Green,499.99,47.10,47.10,15,75,Superloeffel,80,0,14:23 PDT 08-24-2008,0,0,0,0


In [25]:
df10=df.drop(['product_id','item','retail','price','finalprice','bidincrement','bidfee','winner','placedbids','freebids','endtime_str','flg_click_only','flg_beginnerauction','flg_fixedprice','flg_endprice'],axis='columns')
df10

,auction_id,desc
0,86827,Sony Ericsson S500i Unlocked Mysterious Green
1,87964,PSP Slim & Lite Sony Piano Black
2,87965,PSP Slim & Lite Sony Piano Black
3,88638,Sony Ericsson S500i Unlocked Mysterious Green
4,88639,Sony Ericsson S500i Unlocked Mysterious Green
...,...,...
121414,259065,300 Bids Voucher
121415,259066,300 Bids Voucher
121416,259068,300 Bids Voucher
121417,259069,300 Bids Voucher


In [27]:
listid=pd.read_csv('idnumber.csv')
listid

,id
0,224875
1,247448
2,239006
3,231525
4,229731
...,...
312,240738
313,247022
314,252351
315,252353


In [28]:
df11=df10.loc[df10.auction_id.isin(listid.id)]
df11

,auction_id,desc
99379,222661,50 Bids Voucher
99790,223626,Sony PlayStation 3 Slim Console (120GB)
100732,224875,Nokia N97 (Unlocked)
100757,224909,Apple iMac MB418LL/A 24-Inch Desktop
100777,224929,Apple MacBook Pro MB991LL/A 13.3-Inch Laptop
...,...,...
120304,255483,Nintendo DSi Console (Black)
120775,256076,Need For Speed: Shift (Xbox 360)
121099,256602,Wii | Nintendo Console + Wii Sports
121397,259038,50 Bids Voucher


In [29]:
listid = listid.rename(columns={'id': 'auction_id'})
listid

,auction_id
0,224875
1,247448
2,239006
3,231525
4,229731
...,...
312,240738
313,247022
314,252351
315,252353


In [30]:
merged = listid.merge(df11, how='left', indicator=True)
merged

,auction_id,desc,_merge
0,224875,Nokia N97 (Unlocked),both
1,247448,Wii | Nintendo Console + Wii Sports,both
2,239006,WET (PS3),both
3,231525,300 Bids Voucher,both
4,229731,Acer AS8730-6951 18.4-Inch Laptop,both
...,...,...,...
312,240738,Aion: The Tower of Eternity (PC),both
313,247022,Sony PlayStation 3 Slim Console (120GB),both
314,252351,Wii Sports Resort with Wii MotionPlus,both
315,252353,Wii Sports Resort with Wii MotionPlus,both


In [31]:
df11.head(100).to_csv('df11.csv')

In [32]:
gametype=['game','game','game','game','game','game','game','game','game','game',
        'game','game','game','game','game','game','game','game','game','game',
        'game','game','game','game','game','game','game','game','game','game',
        'game','game','game','game','game','game','game','game','game','game',
        'game','game','game','game','game','game','game','game','game','game',
        'game','game','game','game','game','game','game','game','game','game',
        'game','game','game','game','game','game','game','game','game','game',
        'game','game','game','game','game','game','game','game','game','game',
        'game','game','game','game','game','game','game','game','game','game',
        'game','game','game','game','game','game','game','game','game','game',
        'game','game','game','game','game','game','game','game','game','game',
        'game','game','game','game','game','game','game','game','game','game',
        'game','game','game','game','game','game','game','game','game','game',
        'game','game','game','game','game','game','game','game','game','game',
        'game','game','game','game','game','game','game','game','game','game',
        'game','game','game','game','game','game','game','game','game','game',
        'game','game','game','game','game','game','game','game','game','game',
        'game','game','game','game','game','game','game','game','game','game',
        'game','game','game','game','game','game','game','game','game','game',
        'game','game','game','game','game','game','game','game','game','game',
        'game','game','game','game','game','game','game','game','game','game',
        'game','game','game','game','game','game','game','game','game','game',
        'game','game','game','game','game','game','game','game','game','game',
        'game','game','game','game','game','game','game','game','game','game',
        'game','game','game','game','game','game','game','game','game','game',
        'game','game','game','game','game','game','game','game','game','game',
        'game','game','game','game','game','game','game','game','game','game',
        'game','game','game','game','game','game','game','game','game','game',
        'game','game','game','game','game','game','game','game','game','game',
        'game','game','game']
df11['stype']=gametype
df11.head()

<ipython-input-32-e3e15d792c4c>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df11['stype']=gametype


,auction_id,desc,stype
99379,222661,50 Bids Voucher,game
99790,223626,Sony PlayStation 3 Slim Console (120GB),game
100732,224875,Nokia N97 (Unlocked),game
100757,224909,Apple iMac MB418LL/A 24-Inch Desktop,game
100777,224929,Apple MacBook Pro MB991LL/A 13.3-Inch Laptop,game


In [33]:
lifestype=['game','game','game','game','game','game','game','game','game','game',
        'game','game','game','game','game','game','game','game','game','game',
        'game','game','game','game','game','game','game','game','game','game',
        'game','game','game','game','game','game','game','game','game','game',
        'game','game','game','game','game','game','game','game','game','game',
        'game','game','game','game','game','game','game','game','game','game',
        'game','game','game','game','game','game','game','game','game','game',
        'game','game','game','game','game','game','game','game','game','game',
        'game','game','game','game','game','game','game','game','game','game',
        'game','game','game','game','game','game','game','game','game','game',
        'game','game','game','game','game','game','game','game','game','game',
        'game','game','game','game','game','game','game','game','game','game',
        'game','game','game','game','game','game','game','game','game','game',
        'game','game','game','game','game','game','game','game','game','game',
        'game','game','game','game','game','game','game','game','game','game',
        'game','game','game','game','game','game','game','game','game','game',
        'game','game','game','game','game','game','game','game','game','game',
        'game','game','game','game','game','game','game','game','game','game',
        'game','game','game','game','game','game','game','game','game','game',
        'game','game','game','game','game','game','game','game','game','game',
        'game','game','game','game','game','game','game','game','game','game',
        'game','game','game','game','game','game','game','game','game','game',
        'game','game','game','game','game','game','game','game','game','game',
        'game','game','game','game','game','game','game','game','game','game',
        'game','game','game','game','game','game','game','game','game','game',
        'game','game','game','game','game','game','game','game','game','game',
        'game','game','game','game','game','game','game','game','game','game',
        'game','game','game','game','game','game','game','game','game','game',
        'game','game','game','game','game','game','game','game','game','game',
        'game','game','game']
df11['lifestyle']=lifestype
df11.head()

<ipython-input-33-e73830bc0440>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df11['lifestyle']=lifestype


,auction_id,desc,stype,lifestyle
99379,222661,50 Bids Voucher,game,game
99790,223626,Sony PlayStation 3 Slim Console (120GB),game,game
100732,224875,Nokia N97 (Unlocked),game,game
100757,224909,Apple iMac MB418LL/A 24-Inch Desktop,game,game
100777,224929,Apple MacBook Pro MB991LL/A 13.3-Inch Laptop,game,game


In [34]:
df11['reward']=df11['desc']
df11.head()

<ipython-input-34-d326775158d0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df11['reward']=df11['desc']


,auction_id,desc,stype,lifestyle,reward
99379,222661,50 Bids Voucher,game,game,50 Bids Voucher
99790,223626,Sony PlayStation 3 Slim Console (120GB),game,game,Sony PlayStation 3 Slim Console (120GB)
100732,224875,Nokia N97 (Unlocked),game,game,Nokia N97 (Unlocked)
100757,224909,Apple iMac MB418LL/A 24-Inch Desktop,game,game,Apple iMac MB418LL/A 24-Inch Desktop
100777,224929,Apple MacBook Pro MB991LL/A 13.3-Inch Laptop,game,game,Apple MacBook Pro MB991LL/A 13.3-Inch Laptop


In [35]:
linked=['google.com','google.com','google.com','google.com','google.com','google.com','google.com','google.com','google.com','google.com',
        'google.com','google.com','google.com','google.com','google.com','google.com','google.com','google.com','google.com','google.com',
        'google.com','google.com','google.com','google.com','google.com','google.com','google.com','google.com','google.com','google.com',
        'google.com','google.com','google.com','google.com','google.com','google.com','google.com','google.com','google.com','google.com',
        'google.com','google.com','google.com','google.com','google.com','google.com','google.com','google.com','google.com','google.com',
        'google.com','google.com','google.com','google.com','google.com','google.com','google.com','google.com','google.com','google.com',
        'google.com','google.com','google.com','google.com','google.com','google.com','google.com','google.com','google.com','google.com',
        'google.com','google.com','google.com','google.com','google.com','google.com','google.com','google.com','google.com','google.com',
        'google.com','google.com','google.com','google.com','google.com','google.com','google.com','google.com','google.com','google.com',
        'google.com','google.com','google.com','google.com','google.com','google.com','google.com','google.com','google.com','google.com',
        'google.com','google.com','google.com','google.com','google.com','google.com','google.com','google.com','google.com','google.com',
        'google.com','google.com','google.com','google.com','google.com','google.com','google.com','google.com','google.com','google.com',
        'google.com','google.com','google.com','google.com','google.com','google.com','google.com','google.com','google.com','google.com',
        'google.com','google.com','google.com','google.com','google.com','google.com','google.com','google.com','google.com','google.com',
        'google.com','google.com','google.com','google.com','google.com','google.com','google.com','google.com','google.com','google.com',
        'google.com','google.com','google.com','google.com','google.com','google.com','google.com','google.com','google.com','google.com',
        'google.com','google.com','google.com','google.com','google.com','google.com','google.com','google.com','google.com','google.com',
        'google.com','google.com','google.com','google.com','google.com','google.com','google.com','google.com','google.com','google.com',
        'google.com','google.com','google.com','google.com','google.com','google.com','google.com','google.com','google.com','google.com',
        'google.com','google.com','google.com','google.com','google.com','google.com','google.com','google.com','google.com','google.com',
        'google.com','google.com','google.com','google.com','google.com','google.com','google.com','google.com','google.com','google.com',
        'google.com','google.com','google.com','google.com','google.com','google.com','google.com','google.com','google.com','google.com',
        'google.com','google.com','google.com','google.com','google.com','google.com','google.com','google.com','google.com','google.com',
        'google.com','google.com','google.com','google.com','google.com','google.com','google.com','google.com','google.com','google.com',
        'google.com','google.com','google.com','google.com','google.com','google.com','google.com','google.com','google.com','google.com',
        'google.com','google.com','google.com','google.com','google.com','google.com','google.com','google.com','google.com','google.com',
        'google.com','google.com','google.com','google.com','google.com','google.com','google.com','google.com','google.com','google.com',
        'google.com','google.com','google.com','google.com','google.com','google.com','google.com','google.com','google.com','google.com',
        'google.com','google.com','google.com','google.com','google.com','google.com','google.com','google.com','google.com','google.com',
        'google.com','google.com','google.com']
df11['linked']=linked
df11.head()

<ipython-input-35-cd5a907d2a53>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df11['linked']=linked


,auction_id,desc,stype,lifestyle,reward,linked
99379,222661,50 Bids Voucher,game,game,50 Bids Voucher,google.com
99790,223626,Sony PlayStation 3 Slim Console (120GB),game,game,Sony PlayStation 3 Slim Console (120GB),google.com
100732,224875,Nokia N97 (Unlocked),game,game,Nokia N97 (Unlocked),google.com
100757,224909,Apple iMac MB418LL/A 24-Inch Desktop,game,game,Apple iMac MB418LL/A 24-Inch Desktop,google.com
100777,224929,Apple MacBook Pro MB991LL/A 13.3-Inch Laptop,game,game,Apple MacBook Pro MB991LL/A 13.3-Inch Laptop,google.com


In [36]:
bracelet=['[]','[]','[]','[]','[]','[]','[]','[]','[]','[]',
        '[]','[]','[]','[]','[]','[]','[]','[]','[]','[]',
        '[]','[]','[]','[]','[]','[]','[]','[]','[]','[]',
        '[]','[]','[]','[]','[]','[]','[]','[]','[]','[]',
        '[]','[]','[]','[]','[]','[]','[]','[]','[]','[]',
        '[]','[]','[]','[]','[]','[]','[]','[]','[]','[]',
        '[]','[]','[]','[]','[]','[]','[]','[]','[]','[]',
        '[]','[]','[]','[]','[]','[]','[]','[]','[]','[]',
        '[]','[]','[]','[]','[]','[]','[]','[]','[]','[]',
        '[]','[]','[]','[]','[]','[]','[]','[]','[]','[]',
        '[]','[]','[]','[]','[]','[]','[]','[]','[]','[]',
        '[]','[]','[]','[]','[]','[]','[]','[]','[]','[]',
        '[]','[]','[]','[]','[]','[]','[]','[]','[]','[]',
        '[]','[]','[]','[]','[]','[]','[]','[]','[]','[]',
        '[]','[]','[]','[]','[]','[]','[]','[]','[]','[]',
        '[]','[]','[]','[]','[]','[]','[]','[]','[]','[]',
        '[]','[]','[]','[]','[]','[]','[]','[]','[]','[]',
        '[]','[]','[]','[]','[]','[]','[]','[]','[]','[]',
        '[]','[]','[]','[]','[]','[]','[]','[]','[]','[]',
        '[]','[]','[]','[]','[]','[]','[]','[]','[]','[]',
        '[]','[]','[]','[]','[]','[]','[]','[]','[]','[]',
        '[]','[]','[]','[]','[]','[]','[]','[]','[]','[]',
        '[]','[]','[]','[]','[]','[]','[]','[]','[]','[]',
        '[]','[]','[]','[]','[]','[]','[]','[]','[]','[]',
        '[]','[]','[]','[]','[]','[]','[]','[]','[]','[]',
        '[]','[]','[]','[]','[]','[]','[]','[]','[]','[]',
        '[]','[]','[]','[]','[]','[]','[]','[]','[]','[]',
        '[]','[]','[]','[]','[]','[]','[]','[]','[]','[]',
        '[]','[]','[]','[]','[]','[]','[]','[]','[]','[]',
        '[]','[]','[]']
df11['bracelet']=bracelet
df11.head()

<ipython-input-36-298d8021a8a7>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df11['bracelet']=bracelet


,auction_id,desc,stype,lifestyle,reward,linked,bracelet
99379,222661,50 Bids Voucher,game,game,50 Bids Voucher,google.com,[]
99790,223626,Sony PlayStation 3 Slim Console (120GB),game,game,Sony PlayStation 3 Slim Console (120GB),google.com,[]
100732,224875,Nokia N97 (Unlocked),game,game,Nokia N97 (Unlocked),google.com,[]
100757,224909,Apple iMac MB418LL/A 24-Inch Desktop,game,game,Apple iMac MB418LL/A 24-Inch Desktop,google.com,[]
100777,224929,Apple MacBook Pro MB991LL/A 13.3-Inch Laptop,game,game,Apple MacBook Pro MB991LL/A 13.3-Inch Laptop,google.com,[]


In [45]:
df122=pd.read_table('news.tsv')
df122.head()

,N55528,lifestyle,lifestyleroyals,"The Brands Queen Elizabeth, Prince Charles, and Prince Philip Swear By","Shop the notebooks, jackets, and more that the royals can't live without.",https://assets.msn.com/labs/mind/AAGH0ET.html,"[{""Label"": ""Prince Philip, Duke of Edinburgh"", ""Type"": ""P"", ""WikidataId"": ""Q80976"", ""Confidence"": 1.0, ""OccurrenceOffsets"": [48], ""SurfaceForms"": [""Prince Philip""]}, {""Label"": ""Charles, Prince of Wales"", ""Type"": ""P"", ""WikidataId"": ""Q43274"", ""Confidence"": 1.0, ""OccurrenceOffsets"": [28], ""SurfaceForms"": [""Prince Charles""]}, {""Label"": ""Elizabeth II"", ""Type"": ""P"", ""WikidataId"": ""Q9682"", ""Confidence"": 0.97, ""OccurrenceOffsets"": [11], ""SurfaceForms"": [""Queen Elizabeth""]}]",[]
0,N19639,health,weightloss,50 Worst Habits For Belly Fat,These seemingly harmless habits are holding yo...,https://assets.msn.com/labs/mind/AAB19MK.html,"[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik...","[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik..."
1,N61837,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches...,Lt. Ivan Molchanets peeked over a parapet of s...,https://assets.msn.com/labs/mind/AAJgNsz.html,[],"[{""Label"": ""Ukraine"", ""Type"": ""G"", ""WikidataId..."
2,N53526,health,voices,I Was An NBA Wife. Here's How It Affected My M...,"I felt like I was a fraud, and being an NBA wi...",https://assets.msn.com/labs/mind/AACk2N6.html,[],"[{""Label"": ""National Basketball Association"", ..."
3,N38324,health,medical,"How to Get Rid of Skin Tags, According to a De...","They seem harmless, but there's a very good re...",https://assets.msn.com/labs/mind/AAAKEkt.html,"[{""Label"": ""Skin tag"", ""Type"": ""C"", ""WikidataI...","[{""Label"": ""Skin tag"", ""Type"": ""C"", ""WikidataI..."
4,N2073,sports,football_nfl,Should NFL be able to fine players for critici...,Several fines came down against NFL players fo...,https://assets.msn.com/labs/mind/AAJ4lap.html,"[{""Label"": ""National Football League"", ""Type"":...","[{""Label"": ""National Football League"", ""Type"":..."


In [46]:
print(df122['https://assets.msn.com/labs/mind/AAGH0ET.html'].dtype)
print(df122['lifestyleroyals'].dtype)
print(df122['lifestyle'].dtype)
print(df122['[{"Label": "Prince Philip, Duke of Edinburgh", "Type": "P", "WikidataId": "Q80976", "Confidence": 1.0, "OccurrenceOffsets": [48], "SurfaceForms": ["Prince Philip"]}, {"Label": "Charles, Prince of Wales", "Type": "P", "WikidataId": "Q43274", "Confidence": 1.0, "OccurrenceOffsets": [28], "SurfaceForms": ["Prince Charles"]}, {"Label": "Elizabeth II", "Type": "P", "WikidataId": "Q9682", "Confidence": 0.97, "OccurrenceOffsets": [11], "SurfaceForms": ["Queen Elizabeth"]}]'].dtype)

object
object
object
object


In [47]:
print(df11['bracelet'].dtype)
print(df11['lifestyle'].dtype)
print(df11['reward'].dtype)
print(df11['linked'].dtype)

object
object
object
object


In [48]:
df11=df11[['auction_id','stype','lifestyle','reward','desc','linked',	'bracelet']]
df11

,auction_id,stype,lifestyle,reward,desc,linked,bracelet
99379,222661,game,game,50 Bids Voucher,50 Bids Voucher,google.com,[]
99790,223626,game,game,Sony PlayStation 3 Slim Console (120GB),Sony PlayStation 3 Slim Console (120GB),google.com,[]
100732,224875,game,game,Nokia N97 (Unlocked),Nokia N97 (Unlocked),google.com,[]
100757,224909,game,game,Apple iMac MB418LL/A 24-Inch Desktop,Apple iMac MB418LL/A 24-Inch Desktop,google.com,[]
100777,224929,game,game,Apple MacBook Pro MB991LL/A 13.3-Inch Laptop,Apple MacBook Pro MB991LL/A 13.3-Inch Laptop,google.com,[]
...,...,...,...,...,...,...,...
120304,255483,game,game,Nintendo DSi Console (Black),Nintendo DSi Console (Black),google.com,[]
120775,256076,game,game,Need For Speed: Shift (Xbox 360),Need For Speed: Shift (Xbox 360),google.com,[]
121099,256602,game,game,Wii | Nintendo Console + Wii Sports,Wii | Nintendo Console + Wii Sports,google.com,[]
121397,259038,game,game,50 Bids Voucher,50 Bids Voucher,google.com,[]


In [49]:
df11['braceletcopy']=df11['bracelet']
df11

<ipython-input-49-5b4926ffcb7f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df11['braceletcopy']=df11['bracelet']


,auction_id,stype,lifestyle,reward,desc,linked,bracelet,braceletcopy
99379,222661,game,game,50 Bids Voucher,50 Bids Voucher,google.com,[],[]
99790,223626,game,game,Sony PlayStation 3 Slim Console (120GB),Sony PlayStation 3 Slim Console (120GB),google.com,[],[]
100732,224875,game,game,Nokia N97 (Unlocked),Nokia N97 (Unlocked),google.com,[],[]
100757,224909,game,game,Apple iMac MB418LL/A 24-Inch Desktop,Apple iMac MB418LL/A 24-Inch Desktop,google.com,[],[]
100777,224929,game,game,Apple MacBook Pro MB991LL/A 13.3-Inch Laptop,Apple MacBook Pro MB991LL/A 13.3-Inch Laptop,google.com,[],[]
...,...,...,...,...,...,...,...,...
120304,255483,game,game,Nintendo DSi Console (Black),Nintendo DSi Console (Black),google.com,[],[]
120775,256076,game,game,Need For Speed: Shift (Xbox 360),Need For Speed: Shift (Xbox 360),google.com,[],[]
121099,256602,game,game,Wii | Nintendo Console + Wii Sports,Wii | Nintendo Console + Wii Sports,google.com,[],[]
121397,259038,game,game,50 Bids Voucher,50 Bids Voucher,google.com,[],[]


In [59]:
df11.to_csv('df11.tsv',index=False,header=False,sep="\t")

In [60]:
df11.to_csv('df11.csv')

In [ ]:
epochs=1
seed=40
batch_size=64

In [ ]:
MIND_type='demo'

In [ ]:
tmpdir = TemporaryDirectory()
data_path = tmpdir.name

train_news_file = os.path.join(data_path, 'train', df11)
train_behaviors_file = os.path.join(data_path, 'train', impression4)
valid_news_file = os.path.join(data_path, 'valid', df11)
valid_behaviors_file = os.path.join(data_path, 'valid', impression4)
wordEmb_file = os.path.join(data_path, "utils", "embedding.npy")
userDict_file = os.path.join(data_path, "utils", "uid2index.pkl")
wordDict_file = os.path.join(data_path, "utils", "word_dict.pkl")
yaml_file = os.path.join(data_path, "utils", r'lstur.yaml')

mind_url, mind_train_dataset, mind_dev_dataset, mind_utils = get_mind_data_set(MIND_type)

if not os.path.exists(train_news_file):
    download_deeprec_resources(mind_url, os.path.join(data_path, 'train'), mind_train_dataset)

if not os.path.exists(valid_news_file):
    download_deeprec_resources(mind_url, \
                               os.path.join(data_path, 'valid'), mind_dev_dataset)
if not os.path.exists(yaml_file):
    download_deeprec_resources(r'https://recodatasets.z20.web.core.windows.net/newsrec/', \
                               os.path.join(data_path, 'utils'), mind_utils)

TypeError: join() argument must be str, bytes, or os.PathLike object, not 'DataFrame'

In [ ]:
hparams = prepare_hparams(yaml_file,
                          wordEmb_file=wordEmb_file,
                          wordDict_file=wordDict_file,
                          userDict_file=userDict_file,
                          batch_size=batch_size,
                          epochs=epochs)
print(hparams)

HParams object with values {'support_quick_scoring': True, 'dropout': 0.2, 'attention_hidden_dim': 200, 'head_num': 4, 'head_dim': 100, 'filter_num': 400, 'window_size': 3, 'vert_emb_dim': 100, 'subvert_emb_dim': 100, 'gru_unit': 400, 'type': 'ini', 'user_emb_dim': 50, 'learning_rate': 0.0001, 'optimizer': 'adam', 'epochs': 1, 'batch_size': 64, 'show_step': 100000, 'title_size': 30, 'his_size': 50, 'data_format': 'news', 'npratio': 4, 'metrics': ['group_auc', 'mean_mrr', 'ndcg@5;10'], 'word_emb_dim': 300, 'cnn_activation': 'relu', 'model_type': 'lstur', 'loss': 'cross_entropy_loss', 'wordEmb_file': 'C:\\Users\\super\\AppData\\Local\\Temp\\tmp1xlt7xl2\\utils\\embedding.npy', 'wordDict_file': 'C:\\Users\\super\\AppData\\Local\\Temp\\tmp1xlt7xl2\\utils\\word_dict.pkl', 'userDict_file': 'C:\\Users\\super\\AppData\\Local\\Temp\\tmp1xlt7xl2\\utils\\uid2index.pkl'}


In [ ]:
iterator = MINDIterator

In [ ]:
model = LSTURModel(hparams, iterator, seed=seed)

Tensor("conv1d/Relu:0", shape=(None, 30, 400), dtype=float32)
Tensor("att_layer2/Sum_1:0", shape=(None, 400), dtype=float32)


C:\Users\super\anaconda3\envs\python38\lib\site-packages\keras\optimizer_v2\optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [ ]:
print(model.run_eval(df11, impression4))

0it [00:00, ?it/s]


TypeError: __init__(): incompatible constructor arguments. The following argument types are supported:
    1. tensorflow.python.lib.io._pywrap_file_io.BufferedInputStream(filename: str, buffer_size: int, token: tensorflow.python.lib.io._pywrap_file_io.TransactionToken = None)

Invoked with:     auction_id                                           desc
0        86827  Sony Ericsson S500i Unlocked Mysterious Green
1        87964               PSP Slim & Lite Sony Piano Black
2        87965               PSP Slim & Lite Sony Piano Black
3        88638  Sony Ericsson S500i Unlocked Mysterious Green
4        88639  Sony Ericsson S500i Unlocked Mysterious Green
..         ...                                            ...
95       93281                     SanDisk Cruzer Contour 4GB
96       93282                     SanDisk Cruzer Contour 4GB
97       93283                     SanDisk Cruzer Contour 4GB
98       93284                     SanDisk Cruzer Contour 4GB
99       93285            